In [3]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import xesmf as xe
from gfdl_grid_fx import grid_model_sym_approx, regrid_regions_gfdl
from other_grid_fx import generate_reg_grid#, regrid_regions_reg
from plot_fx import make_regional_plots
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from nch import compute_error
import os
#import matplotlib

# matplotlib.use("Agg")  # non-X windows backend

#import matplotlib.pyplot as plt
#import numpy as np
import intake
import sys
import yaml

In [ ]:
cat = intake.open_esm_datastore("/glade/work/netige/mdtf_Nov24/data_catalogs/s_ECMWF-IFS-HR_e_hist-1950_vl_r1i1p1f1_f_mon_v_zos.json")
# filter catalog by desired variable and output frequency
zos_subset = cat.search(variable_id=zos_var, frequency="mon")
# examine assets for a specific file
#zos_subset['CMIP.synthetic.day.r1i1p1f1.day.gr.atmos.r1i1p1f1.1980-01-01-1984-12-31'].df
# convert zos_subset catalog to an xarray dataset dict
zos_dict = zos_subset.to_dataset_dict(
    xarray_open_kwargs={"decode_times": True, "use_cftime": True}
)

In [4]:
cat = intake.open_esm_datastore("/glade/work/netige/mdtf_Nov24/data_catalogs/s_ECMWF-IFS-HR_e_hist-1950_vl_r1i1p1f1_f_mon_v_zos.json")


In [6]:
zos_subset = cat.search(variable_id="zos", frequency="mon")

In [7]:
zos_subset

,unique
activity_id,1
branch_method,1
branch_time_in_child,1
branch_time_in_parent,1
experiment,1
experiment_id,1
frequency,1
grid,1
grid_label,1
institution_id,1


In [8]:
zos_dict = zos_subset.to_dataset_dict(
    xarray_open_kwargs={"decode_times": True, "use_cftime": True}
)
zos_dict


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.frequency.member_id.table_id.grid_label.realm.variant_label'


{'HighResMIP.ECMWF.ECMWF-IFS-HR.hist-1950.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1': <xarray.Dataset>
 Dimensions:             (time: 780, bnds: 2, i: 1021, j: 1442, vertices: 4)
 Coordinates:
   * time                (time) object 1950-01-16 12:00:00 ... 2014-12-16 12:0...
     time_bnds           (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
   * i                   (i) int32 1 2 3 4 5 6 ... 1016 1017 1018 1019 1020 1021
   * j                   (j) int32 1 2 3 4 5 6 ... 1437 1438 1439 1440 1441 1442
     latitude            (i, j) float32 dask.array<chunksize=(511, 721), meta=np.ndarray>
     longitude           (i, j) float32 dask.array<chunksize=(511, 721), meta=np.ndarray>
     vertices_latitude   (i, j, vertices) float32 dask.array<chunksize=(511, 721, 1), meta=np.ndarray>
     vertices_longitude  (i, j, vertices) float32 dask.array<chunksize=(511, 721, 1), meta=np.ndarray>
 Dimensions without coordinates: bnds, vertices
 Data variables:
     zos                 (

In [9]:
key = 'HighResMIP.ECMWF.ECMWF-IFS-HR.hist-1950.mon.r1i1p1f1.Omon.gn.ocean.r1i1p1f1'
dataset = zos_dict[key]

In [10]:
dataset

<xarray.Dataset>
Dimensions:             (time: 780, bnds: 2, i: 1021, j: 1442, vertices: 4)
Coordinates:
  * time                (time) object 1950-01-16 12:00:00 ... 2014-12-16 12:0...
    time_bnds           (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
  * i                   (i) int32 1 2 3 4 5 6 ... 1016 1017 1018 1019 1020 1021
  * j                   (j) int32 1 2 3 4 5 6 ... 1437 1438 1439 1440 1441 1442
    latitude            (i, j) float32 dask.array<chunksize=(511, 721), meta=np.ndarray>
    longitude           (i, j) float32 dask.array<chunksize=(511, 721), meta=np.ndarray>
    vertices_latitude   (i, j, vertices) float32 dask.array<chunksize=(511, 721, 1), meta=np.ndarray>
    vertices_longitude  (i, j, vertices) float32 dask.array<chunksize=(511, 721, 1), meta=np.ndarray>
Dimensions without coordinates: bnds, vertices
Data variables:
    zos                 (time, i, j) float32 dask.array<chunksize=(1, 511, 721), meta=np.ndarray>
Attributes: (12/83)
    Conventions:                             CF-1.7 CMIP-6.0
    activity_id:                             HighResMIP
    branch_method:                           Initialized directly from parent...
    contact:                                 chris.roberts@ecmwf.int
    end_year:                                2014
    experiment:                              coupled historical 1950-2014
    ...                                      ...
    intake_esm_attrs:long_name:              Sea Surface Height Above Geoid
    intake_esm_attrs:units:                  m
    intake_esm_attrs:vertical_levels:        1
    intake_esm_attrs:version:                v0
    intake_esm_attrs:_data_format_:          netcdf
    intake_esm_dataset_key:                  HighResMIP.ECMWF.ECMWF-IFS-HR.hi...